# D⁰ → K⁰ K⁺ K⁻

```{autolink-concat}
```

The decay $D^0 \to K^0K^+K^-$ has a spinless initial and final state, which means that there is no need to align spin with Dalitz-plot decomposition. This notebook shows that the model formulated by {mod}`ampform` is the same as that formulated by {doc}`AmpForm-DPD</index>`. To simplify this comparison, we do not define any dynamics.

In [ ]:
%matplotlib widget

In [ ]:
import logging
import os
import warnings

import ampform
import graphviz
import qrules
from IPython.display import Latex, Markdown

from ampform_dpd import DalitzPlotDecompositionBuilder
from ampform_dpd.adapter.qrules import normalize_state_ids, to_three_body_decay
from ampform_dpd.io import as_markdown_table, aslatex, cached, simplify_latex_rendering

simplify_latex_rendering()
logging.getLogger("jax").setLevel(logging.ERROR)  # mute JAX
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # mute TF
warnings.simplefilter("ignore", category=RuntimeWarning)
if STATIC_PAGE := "EXECUTE_NB" in os.environ:
    logging.getLogger("ampform.sympy").setLevel(logging.ERROR)
    logging.getLogger("ampform_dpd.io").setLevel(logging.ERROR)

## Decay definition

In [ ]:
REACTION = qrules.generate_transitions(
    initial_state="D0",
    final_state=["K0", "K-", "K+"],
    allowed_intermediate_particles=["a(0)", "f(0)(980)", "pi(1)"],
    mass_conservation_factor=0.2,
    formalism="helicity",
)
REACTION123 = normalize_state_ids(REACTION)

In [ ]:
dot = qrules.io.asdot(REACTION123, collapse_graphs=True)
graphviz.Source(dot)

In [ ]:
DECAY = to_three_body_decay(REACTION123.transitions, min_ls=True)
Markdown(as_markdown_table([DECAY.initial_state, *DECAY.final_state.values()]))

In [ ]:
resonances = sorted(
    {t.resonance for t in DECAY.chains},
    key=lambda p: (p.name[0], p.mass),
)
resonance_names = [p.name for p in resonances]
Markdown(as_markdown_table(resonances))

In [ ]:
Latex(aslatex(DECAY, with_jp=True))

## Model formulation

### DPD model

Note that, as opposed to {ref}`Λc⁺ → pπ⁺K⁻<lc2pkpi:Model formulation>` and {ref}`J/ψ → K⁰Σ⁺p̅<jpsi2ksp:Model formulation>`, there are no Wigner-$d$ functions, because the final state is spinless.

In [ ]:
model_builder = DalitzPlotDecompositionBuilder(DECAY, min_ls=True)
DPD_MODEL = model_builder.formulate(cleanup_summations=True)
del model_builder
DPD_MODEL.intensity.cleanup()

In [ ]:
Latex(aslatex(DPD_MODEL.amplitudes, terms_per_line=1))

There is an isobar Wigner-$d$ function, which takes the following helicity angles as argument:

In [ ]:
Latex(aslatex(DPD_MODEL.variables))

### AmpForm model

AmpForm does not formulate alignment Wigner-$D$ functions. For the case of this spinless final state, this means the intensity is the same as that of the [](#dpd-model).

In [ ]:
model_builder = ampform.get_builder(REACTION)
model_builder.use_helicity_couplings = False
model_builder.config.scalar_initial_state_mass = True
model_builder.config.stable_final_state_ids = [0, 1, 2]
AMPFORM_MODEL = model_builder.formulate()
AMPFORM_MODEL.intensity.cleanup()

In [ ]:
Latex(aslatex(AMPFORM_MODEL.amplitudes, terms_per_line=1))

In [ ]:
Latex(aslatex(AMPFORM_MODEL.kinematic_variables))

## Confirm equivalence

In [ ]:
AMPFORM_EXPR = cached.unfold(AMPFORM_MODEL)
AMPFORM_EXPR

In [ ]:
DPD_EXPR = cached.unfold(DPD_MODEL)
DPD_EXPR

In [ ]:
coefficients = {s for s in AMPFORM_MODEL.parameter_defaults if s.name.startswith("C")}
couplings = {s for s in DPD_MODEL.parameter_defaults if "production" in s.name}
substitutions = {s: 1 for s in DPD_MODEL.parameter_defaults if "decay" in s.name}
for c in coefficients:
    resonance_name = c.name[24:].split(";", maxsplit=1)[0]
    resonance_name = resonance_name.rsplit("_", maxsplit=1)[0][:-1]
    coupling, *_ = {s for s in couplings if resonance_name in s.name}
    substitutions[coupling] = c
Latex(aslatex(substitutions))

In [ ]:
assert DPD_EXPR.xreplace(substitutions) == AMPFORM_EXPR